In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ccdv/arxiv-summarization")
dataset = dataset["train"]

dataset = dataset.shuffle(seed=42).select(range(int(0.05 * len(dataset))))

data = dataset.train_test_split(test_size=0.1)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

train-00000-of-00015.parquet:   0%|          | 0.00/230M [00:00<?, ?B/s]

train-00001-of-00015.parquet:   0%|          | 0.00/228M [00:00<?, ?B/s]

train-00002-of-00015.parquet:   0%|          | 0.00/228M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/105M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/105M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/203037 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6436 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [ ]:

print(len(data["train"]))
print(len(data["test"]))


9135
1016


In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
max_input = 512
max_target = 128
batch_size = 3
model_checkpoints = "facebook/bart-large-xsum"

In [ ]:
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained('facebook/bart-large')


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def preprocess(examples):
    model_inputs = tokenizer(
        examples['article'], max_length=1024, truncation=True
    )
    labels = tokenizer(
        text_target=examples['abstract'], max_length=128, truncation=True
    )
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [ ]:
tokenized_data = data.map(preprocess, batched=True)


Map:   0%|          | 0/9135 [00:00<?, ? examples/s]

Map:   0%|          | 0/1016 [00:00<?, ? examples/s]

In [ ]:
!pip install rouge_score
!pip install evaluate


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=995222a287e48d91a1d558c47536a93a20624eff751e1e7b70dd2ae5112f227f
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00


In [ ]:
import evaluate

In [ ]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")


In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result


In [ ]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large')
data_collator = transformers.DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)



pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [ ]:
training_args = transformers.Seq2SeqTrainingArguments(
    output_dir="./bart_finetuning_results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    fp16=True,
    predict_with_generate=True,
    report_to="none",
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = transformers.Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


<ipython-input-24-7d99d89bd337>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Seq2SeqTrainer(


In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:

trainer.train()


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.470000,2.334515,0.175647,0.074144,0.141817,0.161613
2,2.296000,2.305336,0.181691,0.076019,0.144920,0.166272


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=2284, training_loss=2.3972312005305247, metrics={'train_runtime': 730.5683, 'train_samples_per_second': 25.008, 'train_steps_per_second': 3.126, 'total_flos': 4.205266480398336e+16, 'train_loss': 2.3972312005305247, 'epoch': 2.0})

In [ ]:
from datasets import load_dataset

dataset1 = load_dataset("ccdv/arxiv-summarization")
dataset1 = dataset1["test"]

In [ ]:
text_example = data['test']['article'][42]
print(text_example)


in papers [ 1,2 ] have been considered longitudinal vibrations in one - dimensional elastic lumped - parameters line . however in such line vibrations of more general form are possible , if we attribute the idea of one - dimensionality only to general shape of line , not to degree of freedom of vibrating elements of line . 
 this paper investigates the peculiarities of exact analytic solutions for this type of problems . 
 regarding stated above , supposing two degrees of vibration freedom for elastic line elements , the model can be presented in form shown in fig.1 . in case 
 when amplitude of vibration is small ( linear vibration ) , this model can be described by two sets of equations - with respect to @xmath0- and @xmath1-projections of external force affection correspondingly : @xmath2 @xmath3 where @xmath4 is angle of external force inclination to the axe of line . 
 each of these sets of equations is similar to ones having been investigated in [ 1 ] . 
 consequently , we can wr

In [ ]:

input_ids = tokenizer.encode(
    text_example,
    return_tensors="pt",
    max_length=1024,
    truncation=True
).to(device)


In [ ]:
input_ids.shape
summary_text_ids = model.generate(
    input_ids=input_ids,
    bos_token_id=model.config.bos_token_id,
    eos_token_id=model.config.eos_token_id,
    max_length=142,
    min_length=56,
    num_beams=4
)
decoded_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)
print(decoded_text)


in this paper we investigate the peculiarities of exact analytic solutions for one - dimensional elastic lumped - parameters line . 
 it is shown that the most general form of inclined wave in this model can be described by two sets of equations - with respect to the @xmath0- and the angle of external force inclination to the axe of line . in case 
 when amplitude of vibration is small ( linear vibration ) 
 the set of equations describes paramertically the inclined wave propagating along the axe , as shown in diagram of vibration shown in [ 1 ] .    in case of limiting process to distributed -


In [ ]:
text_example = data['test']['abstract'][42]
print(text_example)

in this paper are analysed exact analytic solutions for one - dimensional elastic lumped - parameters line under affection of external force inclined to the axe of line . 
 it is shown that in this case an inclined wave being drscribed by implicit function propagates along the line . 
 the conclusions are extended to unforced vibration in the line and to distributed parameters line vibration . 
 solution in the form of implicit function is proved as a generalizing one for the wave equation .
